# Import libraries

In [2]:
import numpy as np
import tensorflow as tf

# Topics
> ## 1. Build a model
>> ### 1.1 Create placeholder
>>> #### Coding 1: Create integer placeholder

>> ### 1.2 One-hot encoding
>> ### 1.3 Define the weight of CNN
>> ### 1.4 Understanding the blocks/layers
>>> #### Coding 2: Add some layers in the model


> ## 2. Optimization -- Gradient Descent
>> ### 2.1 Cost function and Learning rate
>>> #### Coding 3: Tune the learning rate on cost function

>> ### 2.2 Minibatch

> ## 3. Pre-trained model -- YOLOv3, ResNet (jupyter_2)
> ## 4. Training small dataset, using Tensorboard (jupyter_3)

# 1. Build a model
## 1.1 Create placeholder: 
> ## The pre-allocate dimensions for input and output
![Placeholder](https://imgur.com/lTf4ehx.png)

In [2]:
def create_placeholders(n_H0, n_W0, n_C0, n_y, n_state):
    # Create float placeholder on input_picture
    X = tf.placeholder(tf.float32,[None,n_H0,n_W0,n_C0])
    # Create integer placeholder on output_class
    Y = tf.placeholder(tf.int32,[None,n_y])
    
    ### Coding 1: Create integer placeholder on n_state
    ### START CODE HERE ### (1 line)
    Z = tf.placeholder(tf.int32,[None,n_state])
    #Z = None
    ### END CODE HERE ###
    
    return X, Y, Z

In [3]:
pic_height = 256 #64 #32
pic_width = 256 #64 #32
pic_channel = 3
output_class = 6
input_state = 4

In [4]:
pic_placehold, action_placehold, state_placehold = create_placeholders(pic_height, pic_width, pic_channel, output_class, input_state)
print ("pic_placehold = " + str(pic_placehold))
print ("action_placehold = " + str(action_placehold))
print ("state_placehold = " + str(state_placehold))

pic_placehold = Tensor("Placeholder:0", shape=(?, 256, 256, 3), dtype=float32)
action_placehold = Tensor("Placeholder_1:0", shape=(?, 6), dtype=int32)
state_placehold = Tensor("Placeholder_2:0", shape=(?, 4), dtype=int32)


**Expected Output**

<table> 
<tr>
<td>
    pic_placehold = Tensor("Placeholder:0", shape=(?, 256, 256, 3), dtype=float32)

</td>
</tr>
<tr>
<td>
    action_placehold  = Tensor("Placeholder_1:0", shape=(?, 6), dtype=int32)

</td>
</tr>
<tr>
<th>
    state_placehold  = Tensor("Placeholder_2:0", shape=(?, 4), dtype=int32)
        
</th>
</tr>
</table>

## 1.2 One-hot encoding
> ### ![one-hot](https://imgur.com/jCjGQgi.jpg)

In [26]:
# action = {0:"right", 1:"left", 2:"front", 3:"back", 4:"up", 5:"down"}
action_type = 6

In [33]:
action_onehot = tf.one_hot(action_placehold , action_type)
#print(action_onehot)

## 1.3 Define the weight of CNN: 
> ### Filter size and Channel (Create initialized parameters)
>> ## Filter example
![Sobel filter](https://imgur.com/YaAq4q7.png)
>> ## After filter
![Human](https://imgur.com/C8XqlHm.png)
>> ## Channel
![RGB](https://imgur.com/ygKlPRO.png)

In [26]:
def initialize_parameters():
    # so that your "random" numbers match ours
    tf.set_random_seed(1)                              
    
    # store weight
    # CNN part operation
    # filter_width:4,filter_height:4,input_channel:3,output_channel:8
    W1 = tf.get_variable("W1", [4,4,3,8], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    
    # filter_width:2,filter_height:2,input_channel:8,output_channel:16
    W2 = tf.get_variable("W2", [2,2,8,16], initializer=tf.contrib.layers.xavier_initializer(seed = 0))

    # make it into a dictionary of parameters
    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [78]:
# have a look inside the weight
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters()
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print("W1_first graph_channel values = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2_first graph_channel values = " + str(parameters["W2"].eval()[1,1,1]))

W1_first graph_channel values = [ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
W2_first graph_channel values = [-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]


** Expected Output:**

<table> 

    <tr>
        <td>
        W1_first graph_channel values = 
        </td>
        <td>
[ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394 <br>
 -0.06847463  0.05245192]
        </td>
    </tr>

    <tr>
        <td>
        W2_first graph_channel values = 
        </td>
        <td>
[-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058 <br>
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228 <br>
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]
        </td>
    </tr>

</table>

## 1.4 Understanding the blocks/layers 
> ### CNN: strides, channel
> ### CNN to NN: Flatten()
> ### NN:  Fully_Connected()

> ### Some definitions of functions 
>> (you can check on Tensorflow for other function used: https://www.tensorflow.org/api_docs/python/tf)

> ## Hint for 1.4
- ## <span style="color:red">Graphic operations: CNN</span>
- ### tf.nn.conv2d()
- ### tf.nn.max_pool()
- ### tf.contrib.layers.flatten()
- ### tf.contrib.layers.fully_connected()

In [35]:
def conv_block(tensor_in, channel_in, channel_out, filter_height, filter_width, strides_height, strides_width, maxPool_height=2, maxPool_width=2):
    tf.set_random_seed(1)
    
    w = tf.Variable(tf.truncated_normal([filter_height,filter_width,channel_in,channel_out], stddev=0.1))
    b = tf.Variable(tf.constant(0.1, shape=[channel_out]))
    
    conv = tf.nn.conv2d(tensor_in, w, strides=[1, strides_height, strides_width,1], padding='SAME')                
    print("Conv\t\t%d\t%d\t%d\t\t| %s" % (conv.shape[1],conv.shape[2],conv.shape[3],conv.shape))
    # activate each nodes with bias
    activate_func = tf.nn.relu(conv + b)
    print("ReLU\t\t%d\t%d\t%d\t\t| %s" % (activate_func.shape[1],activate_func.shape[2],activate_func.shape[3],activate_func.shape))
    
    #maxPool_height = 2
    #maxPool_width = 2
    maxPool = tf.nn.max_pool(activate_func, ksize=[1,maxPool_height,maxPool_width,1], strides=[1,maxPool_height,maxPool_width,1], padding='SAME')
    print("max_pool\t%d\t%d\t%d\t\t| %s" %(maxPool.shape[1],maxPool.shape[2],maxPool.shape[3],maxPool.shape))
    return maxPool

In [84]:
s = "---------------------------------------------------------------"
print(s)
print("Layer\t\tHeight\tWidth\tChannel\tNode\t| What you will see")
print(s)
print("Input\t\t%d\t%d\t%d\t\t| %s" %(pic_placehold.shape[1],pic_placehold.shape[2],pic_placehold.shape[3],pic_placehold.shape))

### Coding 2: Add some layers in the model
### Step 1: Tunning the stride for first CNN layer
### Tunning part:
strides_height = 2
strides_width = 2
###
conv1 = conv_block(pic_placehold, 3, 8, 4, 4, strides_height, strides_width)

---------------------------------------------------------------
Layer		Height	Width	Channel	Node	| What you will see
---------------------------------------------------------------
Input		256	256	3		| (?, 256, 256, 3)
Conv		128	128	8		| (?, 128, 128, 8)
ReLU		128	128	8		| (?, 128, 128, 8)
max_pool	64	64	8		| (?, 64, 64, 8)


** Step 1: Expected Output **
<table>
    <tr>
        <td>
            Input | (?, 256, 256, 3)
        </td>
    </tr>
    <tr>
        <td>
            Conv | (?, 128, 128, 8)
        </td>
    </tr>
    <tr>
        <td>
            ReLU | (?, 128, 128, 8)
        </td>
    </tr>
    <tr>
        <td>
            max_pool | (?, 64, 64, 8)
        </td>
    </tr>
</table>

In [82]:
### Step 2: Tunning the stride for second CNN layer + tune the channel
### Tunning part: divisible stride
strides_height = #4
strides_width = #2
channel_in = #8
channel_out = #16
###
conv2 = conv_block(conv1, channel_in, channel_out, 4, 4, strides_height, strides_width)

Conv		16	32	16		| (?, 16, 32, 16)
ReLU		16	32	16		| (?, 16, 32, 16)
max_pool	8	16	16		| (?, 8, 16, 16)


** Step 2: Expected Output **
<table>
    <tr>
        <td>
            Conv | (?, 16, 32, 16)
        </td>
    </tr>
    <tr>
        <td>
            ReLU | (?, 16, 32, 16)
        </td>
    </tr>
    <tr>
        <td>
            max_pool | (?, 8, 16, 16)
        </td>
    </tr>
</table>

In [83]:
### Step 3: Tunning the stride + build the third layer of CNN
### Tunning part: non-divisible stride
strides_height = #2
strides_width = #3
###
#conv3 = conv_block(conv2, 16, 32, 4, 4, strides_height, strides_width)

Conv		4	6	32		| (?, 4, 6, 32)
ReLU		4	6	32		| (?, 4, 6, 32)
max_pool	2	3	32		| (?, 2, 3, 32)


** Step 3: Expected Output **
<table>
    <tr>
        <td>
            Conv | (?, 4, 6, 32)
        </td>
    </tr>
    <tr>
        <td>
            ReLU | (?, 4, 6, 32)
        </td>
    </tr>
    <tr>
        <td>
            max_pool | (?, 2, 3, 32)
        </td>
    </tr>
</table>

## Questions
### 1. How does "Step 1", "Step 2", "Step 3" happens?
> #### CNN operation
>> ![CNN](https://imgur.com/FIy5Ou4.gif)

> #### Max Pooling operation (Vote for the largest number)
>> ![Max Pool](https://imgur.com/ec0zNkC.png)

### 2. Why do we put pic_placehold in "Step 1" conv_block?

In [68]:
def fc_block(tensor_in, node_out, activate_func):
    # We do not care the reusable or not now, so we do not give variable a name
    node_in = int(tensor_in.shape[1])
    w = tf.Variable(tf.random_normal([node_in, node_out], stddev=0.35))
    b = tf.Variable(tf.zeros([node_out]))
    
    if activate_func == None:
        ### 2 way
        #fully_connect = tf.contrib.layers.fully_connected(tensor_in, num_outputs= node_out,activation_fn= None)
        fully_connect = tf.add(tf.matmul(tensor_in,w), b)
        ###
        print("fully_connect\t\t\t\t%d\t| %s" % (fully_connect.shape[1],fully_connect.shape))
    elif activate_func == tf.nn.relu:
        ### 2 way
        #fully_connect = tf.contrib.layers.fully_connected(tensor_in, num_outputs= node_out,activation_fn= activate_func)
        fully_connect = tf.nn.relu(tf.add(tf.matmul(tensor_in,w), b))
        ###
        print("fully_connect\t\t\t\t%d\t| %s" % (fully_connect.shape[1],fully_connect.shape))
        print("ReLU\t\t\t\t\t%d\t| %s" % (fully_connect.shape[1],fully_connect.shape))
    elif activate_func == tf.nn.softmax:
        ### 2 way
        #fully_connect = tf.contrib.layers.fully_connected(tensor_in, num_outputs= node_out,activation_fn= activate_func)
        fully_connect = tf.nn.softmax(tf.add(tf.matmul(tensor_in,w), b))
        ###
        print("fully_connect\t\t\t\t%d\t| %s" % (fully_connect.shape[1],fully_connect.shape))
        print("Softmax\t\t\t\t\t%d\t| %s" % (fully_connect.shape[1],fully_connect.shape))
    
    return fully_connect

In [87]:
# Mix with CNN, NN
s = "---------------------------------------------------------------"
print(s)
print("Layer\t\tHeight\tWidth\tChannel\tNode\t| What you will see")
print(s)
print("Input\t\t%d\t%d\t%d\t\t| %s" %(pic_placehold.shape[1],pic_placehold.shape[2],pic_placehold.shape[3],pic_placehold.shape))

### Step 4: Select max_pool size + Build 2 CNN layer + Select any filter size
#conv1 = conv_block(pic_placehold, 3, 16, 4, 4, 2, 2, 4, 1)
#conv2 = conv_block(conv1, 16, 64, 2, 2, 1, 1)
###

#flat = tf.contrib.layers.flatten(conv2)
flat = tf.reshape(conv2, [-1, int(conv2.shape[1])*int(conv2.shape[2])*int(conv2.shape[3])])
print("flatten\t\t\t\t\t%d\t| %s" %(flat.shape[1],flat.shape))

fc_layer1 = fc_block(flat, 256, None)
fc_layer2 = fc_block(fc_layer1, 128, tf.nn.relu)
predicted_prob = fc_block(fc_layer1, output_class, tf.nn.softmax)

---------------------------------------------------------------
Layer		Height	Width	Channel	Node	| What you will see
---------------------------------------------------------------
Input		256	256	3		| (?, 256, 256, 3)
Conv		128	128	16		| (?, 128, 128, 16)
ReLU		128	128	16		| (?, 128, 128, 16)
max_pool	32	128	16		| (?, 32, 128, 16)
Conv		32	128	64		| (?, 32, 128, 64)
ReLU		32	128	64		| (?, 32, 128, 64)
max_pool	16	64	64		| (?, 16, 64, 64)
flatten					65536	| (?, 65536)
fully_connect				256	| (?, 256)
fully_connect				128	| (?, 128)
ReLU					128	| (?, 128)
fully_connect				6	| (?, 6)
Softmax					6	| (?, 6)


** Step 4: Expected Output **
<table>
    <tr>
        <td>
            Input | (?, 256, 256, 3)
        </td>
    </tr>
    <tr>
        <td>
            Conv | (?, 128, 128, 16)
        </td>
    </tr>
    <tr>
        <td>
            ReLU | (?, 128, 128, 16)
        </td>
    </tr>
    <tr>
        <td>
            max_pool | (?, 32, 128, 16)
        </td>
    </tr>
    <tr>
        <td>
            Conv | (?, 32, 128, 64)
        </td>
    </tr>
    <tr>
        <td>
            ReLU | (?, 32, 128, 64)
        </td>
    </tr>
    <tr>
        <td>
            max_pool | (?, 16, 64, 64)
        </td>
    </tr>
</table>

# 2. Optimization -- Gradient Descent
## 2.1 Cost function and Learning rate
> ### Gradient Descent update the weight with: Learning_rate*slope
> ![Gradient Descent](https://imgur.com/YQzIBuB.png)
>> ### 1. Cost function:
![Optimization](https://imgur.com/LXBjfLb.png)
>> ### 2. Learning rate
![Learning rate](https://imgur.com/J8U8fu9.jpg)

> ## Meaning of cost function
>> ### "預測"和"實際"誤差

> ## Optimization Problems
>> ### 1. 局部極小值
>> ### 2. Saddle point

> ## Overfit

## Question
> ### 3. Check how the cost function looks like on the Internet, and explain how the weight changes

In [38]:
tf.set_random_seed(1)
x = tf.placeholder(tf.float32,[None,1])

### Try w=0,random_uniform for mode = 1,2,3 
w = tf.Variable(0,dtype=tf.float32)
#w = tf.Variable(tf.random_uniform([1], seed=2),dtype=tf.float32)
###

### change different cost function
mode = 3
###

if mode == 1:
    coefficients = np.array([[1.],[-10.],[25.]])
    # operator overloading
    # cost = tf.add(tf.add(w**2,tf.multiply(-10.,w),25))
    cost = x[0]*w**2 + x[1]*w + x[2]
elif mode == 2:
    coefficients = np.array([[2.],[4.],[1.]])
    # 0點沒有斜率
    cost = x[0]*(w**4) + x[1]*(w**3) + x[2]*(w**2)
elif mode == 3:
    coefficients = np.array([[5.],[8.],[3.],[1.]])
    cost = x[0]*(w**6)+ x[1]*(w**5)+ x[2]*(w**2)+ x[3]*w

In [39]:
### Coding 3: tune the learning rate
# play for the value between: 0.9 ~ 0.01
learning_rate = 0.5
###
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()
#first way
'''
session = tf.Session()
session.run(init)
print(session.run(w))
'''
print("--------------------------------------------")
print("lr=%f\tweight\t\tvalue update" %learning_rate)
print("--------------------------------------------")
#second way
with tf.Session() as session:
    # initialize
    session.run(init)
    print("Initialize:\t%f" %session.run(w))
    wf = session.run(w)
    for i in range(100):
        # put input inside
        session.run(train,feed_dict={x:coefficients})
        # print the weight & 差異, think about the curve & the lowest point
        print("Epoch %d:\t%f\t%f" %(i+1,session.run(w),session.run(w)-wf))
        wf = session.run(w)

--------------------------------------------
lr=0.500000	weight		value update
--------------------------------------------
Initialize:	0.000000
Epoch 1:	-0.500000	-0.500000
Epoch 2:	-0.281250	0.218750
Epoch 3:	-0.036244	0.245006
Epoch 4:	-0.427545	-0.391301
Epoch 5:	-0.098899	0.328646
Epoch 6:	-0.303973	-0.205074
Epoch 7:	-0.023879	0.280094
Epoch 8:	-0.452248	-0.428368
Epoch 9:	-0.148364	0.303884
Epoch 10:	-0.211884	-0.063520
Epoch 11:	-0.110137	0.101747
Epoch 12:	-0.282425	-0.172288
Epoch 13:	-0.035442	0.246983
Epoch 14:	-0.429146	-0.393704
Epoch 15:	-0.101721	0.327425
Epoch 16:	-0.298536	-0.196815
Epoch 17:	-0.026219	0.272317
Epoch 18:	-0.447570	-0.421351
Epoch 19:	-0.138015	0.309555
Epoch 20:	-0.230476	-0.092461
Epoch 21:	-0.085726	0.144749
Epoch 22:	-0.329558	-0.243832
Epoch 23:	-0.018489	0.311069
Epoch 24:	-0.463024	-0.444535
Epoch 25:	-0.173991	0.289033
Epoch 26:	-0.167955	0.006036
Epoch 27:	-0.178000	-0.010044
Epoch 28:	-0.161398	0.016602
Epoch 29:	-0.189133	-0.027735
Epoch 30:	